In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm.notebook import tqdm
import os
import json
import psycopg2
from sqlalchemy import create_engine


competition = 'asl-signs'


plt.style.use("ggplot")

In [2]:
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle:
    path = Path('..') / 'input'  / 'competition'
    # !pip install -Uqq fastai
else:
    import zipfile, kaggle
    path = Path.home() / '.data' / 'asl-signs'
    if not path.exists():
        path.mkdir(exist_ok=True)
        kaggle.api.competition_download_cli(competition, path=path)
        zipfile.ZipFile(path / f'{competition}.zip').extractall(path)

In [3]:
train = (pd.read_csv(path / 'train_with_meta.csv')
         .assign(idx=lambda x: x.participant_id.astype(str) + '_' + x.sequence_id.astype(str))
         .set_index('idx')
        )
train.head()

,path,participant_id,sequence_id,sign,cnt_partial_nulls,cnt_partial_nulls_by_frame,total_frames,face,left_hand,pose,right_hand
idx,,,,,,,,,,,
26734_1000035562,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow,0.0,0.0,23.0,23.0,0.0,23.0,11.0
28656_1000106739,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait,0.0,0.0,11.0,11.0,0.0,11.0,2.0
16069_100015657,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud,0.0,0.0,105.0,105.0,28.0,105.0,0.0
25571_1000210073,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird,0.0,0.0,12.0,12.0,0.0,12.0,12.0
62590_1000240708,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie,0.0,0.0,18.0,18.0,0.0,18.0,18.0


In [8]:
cols = ['participant_id', 'sequence_id', 'frame', 'type', 'landmark_index', 'x', 'y', 'z']

In [27]:
with open(Path.home() / '.postgres_creds') as f:
    pg_creds = json.load(f)

conn_str = f"postgresql://{pg_creds['user']}:{pg_creds['password']}@{pg_creds['host']}/{pg_creds['dbname']}"
engine = create_engine(conn_str)

In [ ]:
n = len(train)
# n = 10
errors = list()
for i in tqdm(range(n)):
    record = train.iloc[i]

    pq_path = path / record['path']
    try:
        landmarks = (pd.read_parquet(pq_path)
                     .assign(participant_id=str(record['participant_id']),
                             sequence_id=str(record['sequence_id'])
                            )
                     .reindex(columns=cols)
                     .query("type != 'face'")
                    )
        with engine.connect() as con:
            landmarks.to_sql('landmark_sequences', con=con, if_exists="append", index=False, schema="asl_signs")
    except Exception as e:
        errors.append([record.name, type(e)])

  0%|          | 0/94477 [00:00<?, ?it/s]

In [20]:
record = train.iloc[0]
pq_path = path / record['path']
landmarks = (pd.read_parquet(pq_path)
             .assign(participant_id=str(record['participant_id']),
                     sequence_id=str(record['sequence_id'])
                    )
             .reindex(columns=cols)
             .query("type != 'face'")
            )
landmarks

,participant_id,sequence_id,frame,type,landmark_index,x,y,z
468,26734,1000035562,20,left_hand,0,NaN,NaN,NaN
469,26734,1000035562,20,left_hand,1,NaN,NaN,NaN
470,26734,1000035562,20,left_hand,2,NaN,NaN,NaN
471,26734,1000035562,20,left_hand,3,NaN,NaN,NaN
472,26734,1000035562,20,left_hand,4,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
12484,26734,1000035562,42,right_hand,16,0.001660,0.549574,-0.145409
12485,26734,1000035562,42,right_hand,17,0.042694,0.693116,-0.085307
12486,26734,1000035562,42,right_hand,18,0.006723,0.665044,-0.114017
12487,26734,1000035562,42,right_hand,19,-0.014755,0.643799,-0.123488


In [18]:
train.sequence_id.max()

4294914759

In [26]:
train.sequence_id.astype(str).str.len().unique()

array([10,  9,  8,  7,  6,  5])

In [9]:
landmarks = (pd.read_parquet(pq_path)
             .assign(record_id=record.name,
                     row_id=lambda x: x.record_id + '_' + x.row_id
                    )
             .reindex(columns=cols)
             .sort_values()
             .query("type != 'face'")
            )

,row_id,record_id,frame,type,landmark_index,x,y,z
468,26734_1000035562_20-left_hand-0,26734_1000035562,20,left_hand,0,NaN,NaN,NaN
469,26734_1000035562_20-left_hand-1,26734_1000035562,20,left_hand,1,NaN,NaN,NaN
470,26734_1000035562_20-left_hand-2,26734_1000035562,20,left_hand,2,NaN,NaN,NaN
471,26734_1000035562_20-left_hand-3,26734_1000035562,20,left_hand,3,NaN,NaN,NaN
472,26734_1000035562_20-left_hand-4,26734_1000035562,20,left_hand,4,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
12484,26734_1000035562_42-right_hand-16,26734_1000035562,42,right_hand,16,0.001660,0.549574,-0.145409
12485,26734_1000035562_42-right_hand-17,26734_1000035562,42,right_hand,17,0.042694,0.693116,-0.085307
12486,26734_1000035562_42-right_hand-18,26734_1000035562,42,right_hand,18,0.006723,0.665044,-0.114017
12487,26734_1000035562_42-right_hand-19,26734_1000035562,42,right_hand,19,-0.014755,0.643799,-0.123488


In [34]:
db = create_engine(conn_str)
landmarks.to_sql('landmarks', con=db, if_exists="append", index=False, method="multi", schema="asl_signs")

1725

In [4]:
db = create_engine(conn_str)
db.close()

AttributeError: 'Engine' object has no attribute 'close'